# Testing Notebook

https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783
https://data.sfgov.org/profile/edit/developer_settings
https://dev.socrata.com/foundry/data.sfgov.org/wg3w-h783

# Importing Packages

In [311]:
import pandas as pd
from sodapy import Socrata
import folium
import json
import plotly.express as px

import sys

sys.path.append("..")

import src.config as config

# Setting up the Client and Querying the Database

In [278]:
# Example authenticated client (needed for non-public datasets):
client = Socrata("data.sfgov.org",
                 config.api_key,
                 username=config.username,
                 password=config.password)

In [279]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("wg3w-h783", limit=15000, order="incident_date DESC")

# Processing the Response

In [599]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df['incident_datetime'] = pd.to_datetime(results_df['incident_datetime'])

# Data Engineering - District Specific Information 

In [600]:
district_df = results_df[["incident_datetime", "police_district"]]
district_df["Count"] = 1
district_df["incident_datetime"] = district_df.incident_datetime.dt.date
district_df = district_df.groupby(['incident_datetime','police_district'], as_index=False)['Count'].sum()
district_df.incident_datetime = district_df.incident_datetime.astype(str)

/opt/anaconda3/envs/data_mining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/envs/data_mining/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Data Engineering - All SF Information

In [601]:
total_df = results_df[["incident_datetime"]]
total_df["Count"] = 1
total_df = total_df.groupby(total_df.incident_datetime.dt.date)[['Count']].sum()
total_df = total_df.iloc[1:]
total_df = total_df.reset_index()
total_df.incident_datetime = total_df.incident_datetime.astype(str)

/opt/anaconda3/envs/data_mining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [602]:
# Getting the average
mean = total_df.rolling(5).mean().fillna(method="backfill")

In [603]:
d = datetime.strptime(total_df.incident_datetime.values[-1].replace("-", "/"), "%Y/%m/%d")

In [604]:
rng = pd.date_range(d, periods=3, freq='d')
rng = pd.to_datetime(rng, format='%Y%m%d')
to_append = pd.DataFrame({'incident_datetime': rng}) 
to_append["incident_datetime"] = to_append.incident_datetime.dt.date
to_append = to_append.iloc[1:]

In [605]:
total_df = total_df.append(to_append).fillna("null")

In [606]:
total_df.incident_datetime = total_df.incident_datetime.astype(str)

In [607]:
mean = list(mean.Count.values)
mean.extend(to_append.shape[0] * ["null"])

# Modifying Template to Updated index.html

In [608]:
# Read in the file
with open('../template.html', 'r') as file :
    filedata = file.read()

In [609]:
filedata = filedata.replace('          labels: [], // INCLUDE X AXIS DATES', 
                            '          labels: {}, // INCLUDE X AXIS DATES'.format(
                                list(total_df.incident_datetime.values)))

In [610]:
filedata = filedata.replace('            data: [], // Total', 
                            '            data: {}, // Total'.format(
                                list(total_df.Count.values)))

In [611]:
filedata = filedata.replace('            data: [], // Average', 
                            '            data: {}, // Average'.format(
                                mean))

In [612]:
for i in district_df.police_district.unique():
    if i != "Out of SF":
        district_info = district_df[district_df.police_district == i]
        filedata = filedata.replace('            data: [], // {}'.format(i), 
                                    '            data: {}, // {}'.format(list(district_info.Count.values), i))

In [613]:
# Write the file out again
with open('../index.html', 'w') as file:
    file.write(filedata)

In [614]:
to_show = results_df.copy()

to_show = to_show[["incident_datetime", "police_district", "incident_category", "incident_subcategory", 
                   "incident_description", "resolution", "intersection"]]

to_show.columns = ["Incident Date", "District", "Incident Category", "Subcategory", "Description", "Resolution", "Intersection"]

to_show.head(5).to_html(open('../table.html', 'w'), classes="table", border=0, index=False, justify="center")

In [615]:
with open("../index.html", "r") as f1:
    t1 = f1.readlines()
with open("../table.html", "r") as f2:
    t2 = f2.readlines()

initial = 70
for i in range(0,len(t2)):
    t1.insert(initial, t2[i])
    initial = initial + 1

with open("../index.html", "w") as f2:
    f2.writelines(t1)

In [616]:
with open('../index.html','r') as file:
    filedata = file.read()
    filedata = filedata.replace('  <thead>','  <thead class="thead-dark">')
with open('../index.html','w') as file:
    file.write(filedata)

# Resampling Main Response for 1 Week Average Data

In [617]:
avg_district_df = results_df[["incident_datetime", "police_district"]]
avg_district_df["Count"] = 1
avg_district_df = avg_district_df.set_index(avg_district_df["incident_datetime"]).drop(columns=["incident_datetime"])
avg_district_df = avg_district_df.groupby('police_district').resample('1w').sum()
avg_district_df = avg_district_df.reset_index()

/opt/anaconda3/envs/data_mining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [618]:
latest = pd.DataFrame(columns=avg_district_df.columns)

for i in avg_district_df.police_district.unique():
    latest = latest.append(avg_district_df[avg_district_df.police_district == i].iloc[[-2]])

latest = latest[["police_district", "Count"]]
latest.columns = ["Neighborhood", "Count"]
latest.Neighborhood = latest.Neighborhood.apply(lambda x: x.upper())
latest.Count = latest.Count.astype(int)

# Processing GeoJson

In [619]:
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

with open("../src/sf.geojson") as f:
    data = json.load(f)

for i in range(0,len(data["features"])):
    copy_dict = data["features"][i]
    copy_dict["id"] = copy_dict["properties"]["DISTRICT"]
    data["features"][i] = copy_dict

# Creating Plotly Chloropleth Map

In [620]:
fig = px.choropleth_mapbox(latest, geojson=data, locations='Neighborhood', color='Count',
                           color_continuous_scale="Turbo",
                           range_color=(0, latest.Count.max()),
                           mapbox_style="carto-positron",
                           zoom=11, center = {"lat": latitude, "lon": longitude},
                           opacity=0.5,
                           labels={"Count":"Weekly Crime Rate"}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("../plotly_map.html")

# Creating Plotly Bar Plot

In [621]:
results_df["Count"] = 1

In [622]:
incident_category = results_df.groupby("incident_category")[["Count"]].sum().reset_index().sort_values(by="Count", ascending=False).head(10)

In [623]:
fig = px.bar(incident_category, x='incident_category', y='Count', 
             labels={"incident_category":"Incident Category"}, color="Count", opacity=1.0)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.write_html("../incident_category.html")

# ML

In [624]:
# results_ml = client.get("wg3w-h783", limit=500000, order="incident_date DESC")

In [662]:
# df = pd.DataFrame.from_records(results_ml)

In [664]:
df = pd.read_csv("sfpd_reports.csv")

In [626]:
# df.to_csv("sfpd_reports.csv", index=False)

In [627]:
df['incident_datetime'] = pd.to_datetime(df['incident_datetime'])

df = df[~df['incident_category'].isnull()]
df = df[~df['latitude'].isnull()]
df = df[~df['police_district'].isnull()]

df = df[["incident_datetime", "incident_day_of_week", "police_district", "incident_category"]]

In [628]:
group_by_pd = df.copy()
group_by_pd["Count"] = 1
group_by_pd = group_by_pd.set_index(group_by_pd["incident_datetime"]).drop(columns=["incident_datetime"])
group_by_pd = group_by_pd.groupby(['police_district']).resample('1D').sum()
group_by_pd = group_by_pd.reset_index()

In [647]:
X = group_by_pd.copy()

# Feature engineering

In [648]:
X = pd.concat([X, pd.get_dummies(X["police_district"])], axis=1).drop(columns=["police_district"])

In [649]:
# X['year'] = pd.to_datetime(X['incident_datetime']).dt.year
X['month'] = pd.to_datetime(X['incident_datetime']).dt.month
X['day'] = pd.to_datetime(X['incident_datetime']).dt.day

In [650]:
X = X.iloc[:-10]

In [651]:
y = X[["Count"]]
X = X.drop(['incident_datetime', "Count"], axis=1)

# Using DT

In [652]:
from sklearn.tree import DecisionTreeRegressor

regr_1 = DecisionTreeRegressor()
regr_1.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [653]:
pd_districts = ['Bayview', 'Central', 'Ingleside', 'Mission', 'Northern', 
                'Out of SF', 'Park', 'Richmond', 'Southern', 'Taraval', 
                'Tenderloin']

In [654]:
from datetime import datetime, timedelta

In [655]:
d = datetime.today() - timedelta(days=10)

In [656]:
rng = pd.date_range(d, periods=10+2, freq='d')
rng = pd.to_datetime(rng, format='%Y%m%d')

dates_to_query = pd.DataFrame({ 'Date': rng}) 
dates_to_query["year"] = dates_to_query.Date.dt.year
dates_to_query["day"] = dates_to_query.Date.dt.day
dates_to_query["month"] = dates_to_query.Date.dt.month
dates_to_query["Date"] = dates_to_query.Date.dt.date

In [657]:
predictions_dates = dates_to_query.Date.astype(str).values

In [658]:
def get_prediction_for_dates_df(model, dates):
    for i in pd_districts:
        testing = pd.DataFrame(columns=X.columns)
        data = pd.DataFrame({i: 1, 
#                              "year":dates.year.values, 
                             "month":dates.month.values, 
                             "day":dates.day.values})
        testing = testing.append(data).fillna(0)
        y = model.predict(testing)
        dates[i] = y.astype(int)
    return dates

In [659]:
dates_to_query = get_prediction_for_dates_df(regr_1, dates_to_query)
dates_to_query["total_crime_rate"] = dates_to_query.iloc[:, 4:].sum(axis=1)
dates_to_query = dates_to_query[["Date", "total_crime_rate"]]
dates_to_query.columns = ["incident_datetime", "tcr"]
dates_to_query.incident_datetime = dates_to_query.incident_datetime.astype(str)


ai_predictions = total_df[["incident_datetime"]].iloc[:-2]
ai_predictions = pd.merge(ai_predictions, dates_to_query, on="incident_datetime", how="outer").fillna("null")

In [660]:
ai_predictions.loc[ai_predictions.index[50-11], 'tcr'] = total_df.loc[total_df.index[50-11], 'Count']

In [643]:
# Read in the file
with open('../index.html', 'r') as file :
    filedata = file.read()
    
filedata = filedata.replace('            data: [], // AI', 
                            '            data: {}, // AI'.format(
                                list(ai_predictions.tcr.values)))

# Write the file out again
with open('../index.html', 'w') as file:
    file.write(filedata)